# BM40A1401 GPU Computing

## Erik Kuitunen

### Exercise 1

Import needed libraries.

In [1233]:
#!pip install pycuda
import pycuda.autoinit
import pycuda.driver as cuda
from pycuda.compiler import SourceModule
import numpy as np

#### Task 1
Implement a kernel which takes two vectors A and B and adds them together to form a vector C.

Derfining the kernel.

In [1234]:
modd = SourceModule("""
  __global__ void vector_addition( double* a, double* b, double* c, int n_elem) {

    for ( int i = threadIdx.x + blockIdx.x * blockDim.x; 
          i < n_elem; 
          i += gridDim.x * blockDim.x ) {
            
      c[i] = a[i] + b[i];
    
    }
  }
""")

Create data vectors and initalize thread and block sizes

In [1235]:
N = 10 ** 5
a = np.random.randn(N).astype( float )
b = np.random.randn(N).astype( float )

block_dims = ( 1024, 1, 1 )
grid_dims = ( 64, 1, 1 )

Allocate memory to GPU and copy data to device

In [1236]:
a_gpu = cuda.mem_alloc( a.size * a.dtype.itemsize )
cuda.memcpy_htod( a_gpu, a )

b_gpu = cuda.mem_alloc( a.size * a.dtype.itemsize )
cuda.memcpy_htod( b_gpu, b )

c = np.empty_like(a)
c_gpu = cuda.mem_alloc( a.size * a.dtype.itemsize )

Calling the CUDA kernel.

In [1237]:
kernel = modd.get_function( "vector_addition" )

kernel( a_gpu, b_gpu, c_gpu, np.int32(N), block = block_dims, grid = grid_dims )

Copying the results back to host and verifying the results

In [1238]:
cuda.memcpy_dtoh( c, c_gpu )

c_cpu = a + b

if ( c_cpu == c ).all():
    print( "The vectors are the same." )   
else:
    print( "The vector are not the same. Something is wrong." )

The vectors are the same.


#### Task 2

Implement a kernel which multiplies two matrices together.

Defining the kernel

In [1239]:
modd = SourceModule("""
  __global__ void matrix_multiplication( const float* A, const float* B, float* C, int M, int N, int K) {
    
    int row = threadIdx.y; 
    int col = threadIdx.x;
    float C_elem = 0;
    
    if ( row > K-1 || col > K-1 ) {
      return;
    }
    
    for ( int ii = 0; ii < N; ++ii ) {
        
      float A_elem = A[ row * N + ii ];
      float B_elem = B[ col + K * ii ];

      C_elem += A_elem * B_elem;
    
    }
    
    C[ col + row * K ] = C_elem;
    
  } 
""")

Create data matrices and initalize thread and block sizes

In [1240]:
BLOCK_SIZE = 16

M = 14   # Rows of A and C
N = 15   # Columns of A; rows of B
K = 16   # Columns of B and C

A = np.float32( np.random.rand( M, N ) )
B = np.float32( np.random.rand( N, K ) )

block_dims = ( BLOCK_SIZE, BLOCK_SIZE, 1 )
grid_dims = ( 1, 1, 1 )

Allocate memory and copy data from host to device 

In [1241]:
A_gpu = cuda.mem_alloc( A.nbytes )
cuda.memcpy_htod( A_gpu, A )

B_gpu = cuda.mem_alloc( B.nbytes )
cuda.memcpy_htod( B_gpu, B )

C = np.empty( [ A.shape[0], B.shape[1] ], dtype = np.float32 )
C_gpu = cuda.mem_alloc( C.nbytes )


Calling the CUDA kernel.

In [1242]:
kernel = modd.get_function( "matrix_multiplication" )

kernel( A_gpu, B_gpu, C_gpu, np.int32(M), np.int32(N), np.int32(K),
        block = block_dims, grid = grid_dims )

Copying the results back to host and verifying the results

In [1243]:
cuda.memcpy_dtoh( C, C_gpu )

C_cpu = np.dot( A, B )

C_diff = abs( C_cpu - C )

if ( C_diff.all() < 10 ** -6 ):
    print( "The result matrices are (nearly) the same." )   
else:
    print( "The matrices are not the same. Something is wrong." )

The result matrices are (nearly) the same.


Differences in the order of $ 10^7 $ can be found from the results. Why?




#### Task 3

Extend the kernel from task 2 to use shared memory.

In [1244]:
kernel_code_template = """
  __global__ void matmul_sharedmem( const float* A, const float* B, float* C, int M, int N, int K, int n_tiles ) {
    
    float C_elem = 0;
    
    // Block indices, each block computes submatrix of C, C_sub
    int block_row = blockIdx.y;
    int block_col = blockIdx.x;
    
    // Thread indices. Each thread computes an element of C_sub
    int thread_row = threadIdx.y;
    int thread_col = threadIdx.x;

    // Looping through relevant submatrices to compute C_sub
    for (int ii = 0; ii < n_tiles; ++ii ) { 
    
      // Loading submatrices from global memory
      int linear_ind_A = N * thread_row + thread_col + ii * %(TILE_WIDTH)s + block_row * %(TILE_WIDTH)s * N;
      int linear_ind_B = K * thread_row + thread_col + ii * %(TILE_WIDTH)s * K + block_col * %(TILE_WIDTH)s;
      
      // Shared memory for the submatrices of A and B
      __shared__ float A_sub[ %(TILE_WIDTH)s ][ %(TILE_WIDTH)s ];
      __shared__ float B_sub[ %(TILE_WIDTH)s ][ %(TILE_WIDTH)s ];
      
      // TODO check for threads outside matrix bounds
      
      A_sub[ thread_row ][ thread_col ] = A[ linear_ind_A ];
      B_sub[ thread_row ][ thread_col ] = B[ linear_ind_B ];
      
      __syncthreads();
      
      // Doin the actual multiplication of the submatrices
      for (int kk = 0; kk < %(TILE_WIDTH)s; ++kk) {
        
        float A_sub_elem = A_sub[ thread_row ][ kk ];
        float B_sub_elem = B_sub[ kk ][ thread_col ];
        
        C_elem += A_sub_elem * B_sub_elem;
        
      }
      
      __syncthreads();
    
    }
      
    // Saving the C_elem to matrix C
    int linear_ind_C = block_row * %(TILE_WIDTH)s * K + block_col * %(TILE_WIDTH)s + thread_row * K + thread_col;
    C[ linear_ind_C ] = C_elem;
    
  } 
"""

Create data vectors and initalize thread and block sizes

In [1245]:
TILE_WIDTH = 4

M = 8
N = M
K = M

testarr1 = np.array( [1, 1, 1, 1], dtype = np.float32 )
testarr2 = np.array( [1, 2, 3, 4], dtype = np.float32 )
A = np.array( [ 1*testarr1, 2*testarr1, 3*testarr1, 4*testarr1 ] )
B = np.array( [ testarr2, testarr2, testarr2, testarr2 ] )

A = np.float32( np.random.rand( M, N ) )
B = np.float32( np.random.rand( N, K ) )

gridDim = int( N/TILE_WIDTH )
block_dims = ( TILE_WIDTH, TILE_WIDTH, 1 )
grid_dims = ( gridDim, gridDim, 1 )

Allocate memory and copy data from host to device 

In [1246]:
A_gpu = cuda.mem_alloc( A.nbytes )
cuda.memcpy_htod( A_gpu, A )

B_gpu = cuda.mem_alloc( B.nbytes )
cuda.memcpy_htod( B_gpu, B )

C = np.empty( [ A.shape[0], B.shape[1] ], dtype = np.float32 )
C_gpu = cuda.mem_alloc( C.nbytes )

Specify constant TILE_WIDTH for the kernel. Compile and call the kernel.

In [1247]:
kernel_code = kernel_code_template % {
        'TILE_WIDTH': TILE_WIDTH
        }

# Compile the kernel code
mod = SourceModule( kernel_code )

matrixmul = mod.get_function( "matmul_sharedmem" )

matrixmul( A_gpu, B_gpu, C_gpu, np.int32(M), np.int32(N), np.int32(K), np.int32(2),
          block = block_dims, grid = grid_dims )

Copying the results back to host and verifying the results

In [1248]:
cuda.memcpy_dtoh( C, C_gpu )

C_cpu = np.dot( A, B )

print("CPU: ")
print( C_cpu)

print( "\nGPU: ")
print( C )

C_diff = abs( C_cpu - C )
print( "\nDifference: ")
print( C_diff )

if ( C_diff.all() < 10 ** -6 ):
    print( "The result matrices are (nearly) the same." )   
else:
    print( "The matrices are not the same. Something is wrong." )

CPU: 
[[1.0189475 2.4833114 1.7240088 2.2825298 1.4542924 2.5112624 2.4092796
  1.083016 ]
 [1.5417008 2.323924  1.7720449 2.546152  1.8667597 2.592002  2.2686083
  1.3252709]
 [1.811689  2.2873244 2.4921846 2.9266448 2.3186917 3.2190785 2.462441
  1.6410722]
 [1.909859  3.026258  2.5802743 3.3437405 2.1028512 3.2470243 2.4857454
  1.9692321]
 [0.875985  1.1321583 1.4496667 1.5337021 1.1134462 1.7278705 1.2545614
  0.9610581]
 [1.8450305 2.7292013 2.1018882 3.032506  2.3212109 3.114973  2.6378472
  1.7493572]
 [1.7519978 2.838317  1.9783143 2.8842294 2.4369698 3.2436905 3.1015098
  1.5551019]
 [1.5662793 2.6651707 1.926076  2.6647525 1.8791857 2.734553  2.364956
  1.6191969]]

GPU: 
[[1.0189476 2.4833114 1.7240087 2.2825298 1.4542924 2.5112624 2.4092796
  1.0830162]
 [1.5417008 2.323924  1.772045  2.5461524 1.8667597 2.5920024 2.268608
  1.325271 ]
 [1.811689  2.2873242 2.4921846 2.9266448 2.318692  3.2190785 2.462441
  1.641072 ]
 [1.9098588 3.026258  2.5802746 3.3437402 2.1028512 3.2

Once again differences are appearing. Seems to be related to the ratio of the size of matrix and tile width?